In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector

# MySQL Database connection
def connect_db():
    global conn, cursor
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",  # Replace with your MySQL username
            password="root",  # Replace with your MySQL password
            database="contact_book"
        )
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS contacts (
                            id INT AUTO_INCREMENT PRIMARY KEY,
                            name VARCHAR(255) NOT NULL,
                            phone VARCHAR(20) NOT NULL,
                            email VARCHAR(255),
                            Address VARCHAR(200)
                        )''')
        conn.commit()
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")

# Add a new contact
def add_contact():
    if name_var.get() == "" or phone_var.get() == "":
        messagebox.showerror("Error", "Name and Phone are required fields")
        return
    cursor.execute("INSERT INTO contacts (name, phone, email, address) VALUES (%s, %s, %s, %s)",
                   (name_var.get(), phone_var.get(), email_var.get(), address_var.get()))
    conn.commit()
    clear_inputs()
    show_contacts()

# View all contacts
def show_contacts():
    contact_list.delete(0, tk.END)
    cursor.execute("SELECT * FROM contacts")
    rows = cursor.fetchall()
    for row in rows:
        contact_list.insert(tk.END, row)

# Update a contact
def update_contact():
    try:
        selected = contact_list.get(contact_list.curselection())
        contact_id = selected[0]
    except:
        messagebox.showerror("Error", "Please select a contact to update")
        return

    cursor.execute("UPDATE contacts SET name=%s, phone=%s, email=%s, address=%s WHERE id=%s",
                   (name_var.get(), phone_var.get(), email_var.get(), address_var.get(), contact_id))
    conn.commit()
    clear_inputs()
    show_contacts()

# Delete a contact
def delete_contact():
    try:
        selected = contact_list.get(contact_list.curselection())
        contact_id = selected[0]
    except:
        messagebox.showerror("Error", "Please select a contact to delete")
        return

    cursor.execute("DELETE FROM contacts WHERE id=%s", (contact_id,))
    conn.commit()
    clear_inputs()
    show_contacts()

# Clear input fields
def clear_inputs():
    name_var.set("")
    phone_var.set("")
    email_var.set("")
    address_var.set("")

# Select a contact from the Listbox
def select_contact(event):
    try:
        selected = contact_list.get(contact_list.curselection())
        name_var.set(selected[1])
        phone_var.set(selected[2])
        email_var.set(selected[3])
        address_var.set(selected[4])
    except:
        pass

# GUI setup
root = tk.Tk()
root.title("Contact Book")

# Variables
name_var = tk.StringVar()
phone_var = tk.StringVar()
email_var = tk.StringVar()
address_var = tk.StringVar()

# Labels and Entries
tk.Label(root, text="Name:").grid(row=0, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=name_var).grid(row=0, column=1, padx=10, pady=10)

tk.Label(root, text="Phone:").grid(row=1, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=phone_var).grid(row=1, column=1, padx=10, pady=10)

tk.Label(root, text="Email:").grid(row=2, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=email_var).grid(row=2, column=1, padx=10, pady=10)

tk.Label(root, text="Address:").grid(row=3, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=address_var).grid(row=3, column=1, padx=10, pady=10)

# Buttons
tk.Button(root, text="Add", command=add_contact).grid(row=4, column=0, padx=10, pady=10)
tk.Button(root, text="Update", command=update_contact).grid(row=4, column=1, padx=10, pady=10)
tk.Button(root, text="Delete", command=delete_contact).grid(row=4, column=2, padx=10, pady=10)
tk.Button(root, text="Clear", command=clear_inputs).grid(row=4, column=3, padx=10, pady=10)

# Contact Listbox 
contact_list = tk.Listbox(root, height=10, width=50)
contact_list.grid(row=5, column=0, columnspan=4, padx=10, pady=10)
contact_list.bind('<<ListboxSelect>>', select_contact)

# Initialize database and load contacts
connect_db()
show_contacts()

# Start the GUI main loop
root.mainloop()